In [1]:
import os
import numpy as np
import astropy as ap
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon
from astropy.coordinates.angle_utilities import angular_separation
import astropy.units as u
from astropy.time import Time
from astroplan import Observer
from datetime import datetime, timedelta
from collections import namedtuple, OrderedDict

In [2]:
ob = {
                "metadata": {
                "name": "standard stars #9",
                "pi_id": 7766,
                "sem_id": "2021B_K0008",
                "instrument": "KCWI",
                "comment": "I'm a scholar. I enjoy scholarly pursuits."
            },
            "version": "0.1",
            "sequences": [{
                "name": "completed",
                "version": "0.1",
                "det1_exptime": 1637,
                "det1_nexp": 88,
                "det2_exptime": 3185,
                "det2_nexp": 94,
                "cfg_cam_grating": "RL",
                "cfg_cam_cwave": 7581,
                "cfg_slicer": "Medium"
            }],
            "acquisition":{"name":"KCWI_ifu_acq_direct",
            "guider_gs_dec":30.2401,
            "guider_gs_mode":
            "User",
            "guider_gs_ra":3.5337,
            "guider_po":"IFU",
            "script":"KCWI_ifu_acq_direct",
            "version":"0.1"},
            "associations": [
                "jrai",
                "bqtm",
                "hcon"
            ],
            "priority": 28.82745766552508,
            "status": {
                "state": "inqueue",
                "executions": [
                "2019-06-07 03:07:32",
                "2018-02-16 08:38:32",
                ]
            },
            "target" : {
                "name" : "rxmy",
                "ra" : "17:31:16",
                "dec" : "+33:27:43",
                "equinox" : "J2000",
                "frame" : "sghi",
                "ra_offset" : 1.6936631786804168,
                "dec_offset" : 7.48251275931961,
                "pa" : 7.085909143633636,
                "pm_ra" : 8.70563085605572,
                "pm_dec" : 0.6143121450706313,
                "epoch" : 2000,
                "obstime" : "2021-04-22 15:08:04",
                "magnitude" : [
                    {
                        "band" : "V",
                        "magnitude" : 0.9799051741031348
                    },
                    {
                        "band" : "I",
                        "magnitude" : 0.7003428020962619
                    }
                ],
                "wrap" : "north",
                "d_ra" : 0.12075887611724401,
                "d_dec" : 6.329640885945703,
                "comment" : "I hear the jury?s still out on science."
            },
            "_id": "0"
            }

In [3]:
tup = ("A_SCRIPT_NAME", "SCRIPT_STATEMENT", "VERSION 1")
tupScripts = [tup] # immutable object, does not inclue names, array preserves order
name = tupScripts[0][0]
a = [print(tup[0]) for tup in tupScripts]

A_SCRIPT_NAME


In [4]:
Script = namedtuple('Script', ['name', 'script', 'version']) # immutable object, includes names and expected structure

In [5]:
namedTup = Script("A_SCRIPT_NAME", "SCRIPT", "VERSION 1")
namedTupleScripts = [ namedTup ]# array preserves order
name = namedTupleScripts[0].name # gets item by index followed by key
a = [print(x.name) for x in namedTupleScripts]

A_SCRIPT_NAME


In [6]:
od = {"A_SCRIPT_NAME": namedTup} # Mutable object
orderedDictScripts = OrderedDict(od) # Order is perserved
name = orderedDictScripts.get("A_SCRIPT_NAME").name # get script item by dict key followed by named tuple key.
a = [print(script.name) for key, script in orderedDictScripts.items()]

A_SCRIPT_NAME


In [8]:
target = ob.get('target')

In [9]:
ra, dec = target.get('ra'), target.get('dec')
print(f'{ra}, {dec}')

17:31:16, +33:27:43


In [10]:
keck = EarthLocation.of_site('Keck Observatory')
keckObs = Observer.at_site("Keck Observatory", timezone="US/Hawaii")

In [11]:
tgt = SkyCoord(ra=ra, dec=dec, frame='icrs', unit=(u.hourangle, u.deg))
hourOffset = -10
utcoffset = hourOffset * u.hour  # Hawaii Daylight Time
now = datetime.now() - timedelta(hours=hourOffset)
time = Time(now)
nearestSet = keckObs.sun_set_time(time, which='nearest')

todayMidnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
if time < nearestSet: # today's sunrise already occured
    print('using today\'s midnight')
    midnight = Time(todayMidnight) - utcoffset
else: # today's sunrise has not risen yet
    print('using yesterday\'s midnight')
    yesterdayMidnight = todayMidnight - timedelta(days=1)
    midnight = Time(now.replace()) - utcoffset

deltaMidnight = np.linspace(-5, 5, 100)*u.hour
times = midnight+deltaMidnight
tgtFrame = AltAz(obstime=times, location=keck)
tgtTraj = tgt.transform_to(tgtFrame)
tgtAirmass = tgtTraj.secz



using today's midnight


In [12]:
DEG_TO_RAD = np.pi/180
to_rad = lambda x: DEG_TO_RAD * x

In [13]:
moonTraj = get_moon(times, location=keck).transform_to(tgtFrame)
latLngTgtMoon = list(zip(tgtTraj.az, tgtTraj.alt, moonTraj.az, moonTraj.alt))
latLngTgtMoon = [ [x.radian for x in sublist] for sublist in latLngTgtMoon ]
angleOfSeparation = [angular_separation(*x) / DEG_TO_RAD for x in latLngTgtMoon] # in deg
parallaticAngle = keckObs.parallactic_angle(times, tgt).deg # in deg

In [100]:
parallaticAngle

array([108.64515271, 107.16299551, 105.75867637, 104.42406007,
       103.15196648, 101.93604486, 100.77066472,  99.65082123,
        98.57205358,  97.53037464,  96.5222103 ,  95.54434719,
        94.59388754,  93.66821021,  92.76493697,  91.88190323,
        91.01713265,  90.16881501,  89.33528688,  88.51501478,
        87.70658036,  86.90866735,  86.12005013,  85.33958358,
        84.56619412,  83.79887171,  83.03666277,  82.27866383,
        81.52401587,  80.77189923,  80.02152897,  79.27215082,
        78.52303733,  77.77348457,  77.02280896,  76.27034449,
        75.51544015,  74.75745753,  73.99576867,  73.2297541 ,
        72.45880095,  71.68230133,  70.89965071,  70.11024658,
        69.31348705,  68.50876976,  67.69549072,  66.87304338,
        66.04081777,  65.19819975,  64.34457038,  63.47930539,
        62.60177485,  61.71134286,  60.80736751,  59.8892009 ,
        58.95618942,  58.00767415,  57.0429915 ,  56.06147408,
        55.06245185,  54.04525346,  53.00920797,  51.95

In [14]:
tgtFrame

<AltAz Frame (obstime=[datetime.datetime(2021, 10, 8, 5, 0)
 datetime.datetime(2021, 10, 8, 5, 6, 3, 636364)
 datetime.datetime(2021, 10, 8, 5, 12, 7, 272727)
 datetime.datetime(2021, 10, 8, 5, 18, 10, 909091)
 datetime.datetime(2021, 10, 8, 5, 24, 14, 545455)
 datetime.datetime(2021, 10, 8, 5, 30, 18, 181818)
 datetime.datetime(2021, 10, 8, 5, 36, 21, 818182)
 datetime.datetime(2021, 10, 8, 5, 42, 25, 454545)
 datetime.datetime(2021, 10, 8, 5, 48, 29, 90909)
 datetime.datetime(2021, 10, 8, 5, 54, 32, 727273)
 datetime.datetime(2021, 10, 8, 6, 0, 36, 363636)
 datetime.datetime(2021, 10, 8, 6, 6, 40)
 datetime.datetime(2021, 10, 8, 6, 12, 43, 636364)
 datetime.datetime(2021, 10, 8, 6, 18, 47, 272727)
 datetime.datetime(2021, 10, 8, 6, 24, 50, 909091)
 datetime.datetime(2021, 10, 8, 6, 30, 54, 545455)
 datetime.datetime(2021, 10, 8, 6, 36, 58, 181818)
 datetime.datetime(2021, 10, 8, 6, 43, 1, 818182)
 datetime.datetime(2021, 10, 8, 6, 49, 5, 454545)
 datetime.datetime(2021, 10, 8, 6, 55,

In [80]:
delta_midnight = np.linspace(-5, 5, 100)*u.hour

True

In [73]:
endTime

'2021-10-06 04:02:32.685'

In [76]:
startTime

<Time object: scale='utc' format='jd' value=2459493.1787716127>

In [79]:
datetime.now()

# if sun hasn't risen use previous day's midnight
midnight = Time('2012-7-13 00:00:00') - utcoffset
# if sun has risen, use next day's mignight

9